In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchvision import datasets, transforms, models
from torch.autograd import Variable

# load data

In [2]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])


train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('./data', train=True, download=False,
                   transform=transforms.Compose([
                       transforms.RandomResizedCrop(max((224, 224))),
                       transforms.RandomHorizontalFlip(),
                       transforms.ToTensor(),
                       normalize
                   ])),
    batch_size=65, num_workers=2, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('./data', train=False,
                   transform=transforms.Compose([
                       transforms.Resize(int(max((224, 224))/224*256)),
                       transforms.CenterCrop(max((224, 224))),
                       transforms.ToTensor(),
                       normalize
                   ])),
    batch_size=65, num_workers=2, shuffle=True)

# train

In [3]:
class IdentityLayers_C(nn.Module):
    def __init__(self, in_channels, out_channels, expansion,
                 cardinality=32, stride=1, shortcut=None):
        super(IdentityLayers_C, self).__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(in_channels,
                                             out_channels,
                                             kernel_size=1,
                                             bias=False),
                                   nn.BatchNorm2d(out_channels),
                                   nn.ReLU(inplace=True))
        
        self.conv2 = nn.Sequential(nn.Conv2d(out_channels,
                                             out_channels, kernel_size=3,
                                             stride=stride,
                                             padding=1,
                                             groups=cardinality,
                                             bias=False),
                                   nn.BatchNorm2d(out_channels),
                                   nn.ReLU(inplace=True))
        
        self.conv3 = nn.Sequential(nn.Conv2d(out_channels,
                                             out_channels*expansion,
                                             kernel_size=1,
                                             bias=False),
                        nn.BatchNorm2d(out_channels*expansion))

        self.shortcut = shortcut
        
        
    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out)

        if self.shortcut is not None:
            out += self.shortcut(x)
        else:
            out += x

        return F.relu(out, inplace=True)

In [4]:
class ResNeXt(nn.Module):
    def __init__(self, in_channels=3, num_classes=1000, cardinality=32, expansion=2):
        super(ResNeXt, self).__init__()
        self.cardinality = 32
        self.expansion = expansion
        self.conv = nn.Conv2d(in_channels, 64, kernel_size=3,
                              stride=1, padding=1, bias=False)
        self.bn = nn.BatchNorm2d(64)

        self.in_channels = 64
        self.residual1 = self._make_indentity_block(128, 3)
        self.residual2 = self._make_indentity_block(256, 4, stride=2)
        self.residual3 = self._make_indentity_block(512, 6, stride=2)
        self.residual4 = self._make_indentity_block(1024, 3, stride=2)

        self.fc = nn.Linear(1024 * self.expansion, num_classes)

    def _make_indentity_block(self, out_channels, num_layers, stride=1):
        shortcut = None
        layers = []

        if stride != 1 or self.in_channels != out_channels * self.expansion:
            shortcut = nn.Sequential(nn.Conv2d(self.in_channels,
                                               out_channels * self.expansion,
                                               kernel_size=1,
                                               stride=stride,
                                               bias=False),
                                     nn.BatchNorm2d(out_channels * self.expansion))

        layers.append(IdentityLayers_C(self.in_channels, out_channels,
                                       expansion=self.expansion,
                                       stride=stride, shortcut=shortcut))
        self.in_channels = out_channels * self.expansion

        for i in range(1, num_layers):
            layers.append(IdentityLayers_C(self.in_channels, out_channels, expansion=self.expansion))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = F.relu(x, inplace=True)
#         x = F.max_pool2d(x, kernel_size=3, stride=2, padding=1)

        x = self.residual1(x)
        x = self.residual2(x)
        x = self.residual3(x)
        x = self.residual4(x)

        x = F.avg_pool2d(x, kernel_size=4)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [5]:
model = ResNeXt(cardinality=2)
model

ResNeXt(
  (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (residual1): Sequential(
    (0): IdentityLayers_C(
      (conv1): Sequential(
        (0): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
        (2): ReLU(inplace)
      )
      (conv2): Sequential(
        (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
        (2): ReLU(inplace)
      )
      (conv3): Sequential(
        (0): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
      )
      (shortcut): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=

In [6]:
x, y = iter(train_loader).next()
b_x, b_y = Variable(x), Variable(y)

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()
model.train()
optimizer.zero_grad()
outputs = model(b_x)

In [ ]:
loss = loss_func(outputs, b_y)
loss